# Load and use model in Redis AI

1. docker run -p 6379:6379 -it --rm redisai/redisai


* https://oss.redislabs.com/redisai/commands/
* https://oss.redislabs.com/redisai/

Use Redisai.py to upload the model to Redis.

In [1]:
import redisai
import redis
import ml2rt
from PIL import Image
from torchvision import transforms
import numpy as np

class Object(object): pass

In [2]:
tr = Object()

In [3]:
tr.redis = redisai.Client()

In [4]:
tr.loaded_model = ml2rt.load_model('plant.pt')

In [5]:
len(tr.loaded_model)

94481061

In [7]:
tr.redis.modelset(
    "plant", 
    redisai.Backend.torch,
    redisai.Device.cpu,
    tr.loaded_model)

AttributeError: module 'redisai' has no attribute 'Backend'

This is prediction is from the Linear layer. To convert to probabilities, we need to apply the Softmax function:

In [ ]:
tr.conn = redis.Redis()

## Set up Gears

In [ ]:
# !ls ../../test-loop/

In [ ]:
print('Loading gear - ', end='')
with open('../../test-loop/gear_stream.py', 'rb') as tr.f:
    tr.gear = tr.f.read()
    tr.res = tr.conn.execute_command(
        'RG.PYEXECUTE',tr. gear,
        'REQUIREMENTS', 'imageio', 'opencv-python')
    print(tr.res)

In [ ]:
result = tr.conn.execute_command(
    'xadd', 
    'input:0', 
    '*',
    'url', 
    'https://pbs.twimg.com/media/EXKlWjmU8AAlXFh?format=jpg&name=medium',
)

print(result)

In [ ]:
print(tr.conn.execute_command('GET', 'result'))
print(tr.conn.execute_command('GET', 'log:0').decode())

### Twitter Loop

In [ ]:
import io, urllib.request, time, twitter, os, PIL

def twitter_loop():
    msg = Object()
    msg.running = True
    
    api = twitter.Api(
        consumer_key=os.environ['TWITTER_CONSUMER_KEY'],
        consumer_secret=os.environ['TWITTER_CONSUMER_SECRET'],
        access_token_key=os.environ['TWITTER_TOKEN_KEY'],
        access_token_secret=os.environ['TWITTER_TOKEN_SECRET'])   
    
    labels = ['healthy', 'multiple_diseases', 'rust', 'scab']
    
    timeline = api.GetUserTimeline(screen_name="alexonsoftware", 
                        count=10, 
                        include_rts=False,
                        exclude_replies=True)    
    
    since_id = since_id = timeline[0].id \
        if len(timeline) > 0 \
        else 0

    while msg.running:
        msg.t = api.GetUserTimeline(screen_name="alexonsoftware", 
                            count=10, 
                            include_rts=False,
                            exclude_replies=True,
                            since_id=since_id)
        if len(msg.t) > 0:
            for msg.tweet in msg.t:
                if 'media' in msg.tweet.AsDict():
                    msg.media_list = msg.tweet.AsDict()['media']
                    for msg.pic in msg.media_list:
                        
                        result = conn.execute_command(
                            'xadd', 'input:0', '*',
                            'url', msg.pic['media_url'])
                        print(result)
                        api.PostUpdate(
                            status=result,
                            in_reply_to_status_id=msg.tweet.id
                        )
                since_id = max(since_id, msg.tweet.id)
        time.sleep(1)
        msg.running = True

In [ ]:
twitter_loop()